In [ ]:
pip install tensorflow

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
train_generator = datagen.flow_from_directory('data/train', target_size=(224, 224), batch_size=32, class_mode='categorical')


In [ ]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=25, validation_data=val_generator)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(test_generator)
print(classification_report(y_true, y_pred))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import plot_model

# Load the pre-trained model
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Visualize the model architecture
plot_model(model, to_file='vgg16_model.png', show_shapes=True, show_layer_names=True)


In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# One-hot encode the labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True)
datagen.fit(train_images)


In [ ]:
from tensorflow.keras import models, layers

# Add custom layers on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))  # Assuming 10 classes for CIFAR-10


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    validation_data=(test_images, test_labels),
                    epochs=20)


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy and loss over epochs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()


In [ ]:
import numpy as np
import cv2

# Predict on test images
predictions = model.predict(test_images)

# Function to display images with predicted labels
def display_predictions(images, labels, predictions, class_names):
    for i in range(5):  # Display first 5 images
        plt.figure()
        plt.imshow(images[i])
        plt.title(f'Actual: {class_names[np.argmax(labels[i])]} | Predicted: {class_names[np.argmax(predictions[i])]}')
        plt.show()

# Class names in CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
display_predictions(test_images, test_labels, predictions, class_names)


In [ ]:
import cv2

cap = cv2.VideoCapture(0)  # Open webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the frame for the model
    img = cv2.resize(frame, (224, 224))  # Resize frame to model input size
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    
    # Predict using the pre-trained model
    pred = model.predict(img)
    class_name = class_names[np.argmax(pred)]
    
    # Display the prediction on the frame
    cv2.putText(frame, class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('Real-time Image Classification', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
